In [1]:
import numpy as np
from catboost import CatBoostRegressor, FeaturesData, Pool
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
train= pd.read_csv('train.csv',nrows=1000000)
train.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400024,3.150018,18.0,92.0


In [3]:
dev_in= pd.read_csv('dev_in.csv',nrows=1000000)
dev_in

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.165300,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1549149600,50.459202,4.453820,1.0,10,mild temperate,167.0,-54.973902,3.760714,743.401115,...,0.11758,10.609865,0.0,0.000000,0.0,0.0,-0.500000,-0.650000,0.0,100.0
49996,1550149200,14.865600,-91.501999,2.0,10,mild temperate,2530.0,9.415068,11.627143,604.499035,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.672150,14.188867,0.0,0.0
49997,1553637300,37.688999,-121.442001,18.0,10,mild temperate,56.0,52.537946,12.823571,743.326800,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-2.497864,4.249994,-32.0,63.0
49998,1548885600,27.250999,2.512020,11.0,0,dry,269.0,-62.773181,17.074286,737.590379,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.235107,-5.885107,0.0,0.0


In [4]:
dev_out= pd.read_csv('dev_in.csv',nrows=1000000)
dev_out

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.165300,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1549149600,50.459202,4.453820,1.0,10,mild temperate,167.0,-54.973902,3.760714,743.401115,...,0.11758,10.609865,0.0,0.000000,0.0,0.0,-0.500000,-0.650000,0.0,100.0
49996,1550149200,14.865600,-91.501999,2.0,10,mild temperate,2530.0,9.415068,11.627143,604.499035,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.672150,14.188867,0.0,0.0
49997,1553637300,37.688999,-121.442001,18.0,10,mild temperate,56.0,52.537946,12.823571,743.326800,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-2.497864,4.249994,-32.0,63.0
49998,1548885600,27.250999,2.512020,11.0,0,dry,269.0,-62.773181,17.074286,737.590379,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.235107,-5.885107,0.0,0.0


In [5]:
train.fillna(0)

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400024,3.150018,18.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.549181e+09,-20.434700,16.660801,25.0,0.0,dry,1478.0,48.750396,18.624286,644.911570,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.638489,14.271875,0.0,0.0
999996,1.548596e+09,36.235199,59.640999,13.0,0.0,dry,993.0,1.101464,9.299286,670.941895,...,0.0,0.0,0.0,0.0,0.0,0.0,0.800018,-3.949988,0.0,0.0
999997,1.540392e+09,31.562000,-91.506500,16.0,0.0,mild temperate,18.0,16.050624,15.687143,761.587365,...,0.0,0.0,0.0,0.0,0.0,0.0,0.100006,9.650018,0.0,0.0
999998,1.548875e+09,-14.816000,-39.033199,30.0,10.0,tropical,4.0,26.872053,28.710000,757.429851,...,0.0,0.0,0.0,0.0,0.0,0.0,0.797729,21.047754,-5.0,0.0


In [6]:
dev_in.fillna(0)

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.165300,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1549149600,50.459202,4.453820,1.0,10,mild temperate,167.0,-54.973902,3.760714,743.401115,...,0.11758,10.609865,0.0,0.000000,0.0,0.0,-0.500000,-0.650000,0.0,100.0
49996,1550149200,14.865600,-91.501999,2.0,10,mild temperate,2530.0,9.415068,11.627143,604.499035,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.672150,14.188867,0.0,0.0
49997,1553637300,37.688999,-121.442001,18.0,10,mild temperate,56.0,52.537946,12.823571,743.326800,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-2.497864,4.249994,-32.0,63.0
49998,1548885600,27.250999,2.512020,11.0,0,dry,269.0,-62.773181,17.074286,737.590379,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.235107,-5.885107,0.0,0.0


In [7]:
dev_out.fillna(0)

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.165300,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1549149600,50.459202,4.453820,1.0,10,mild temperate,167.0,-54.973902,3.760714,743.401115,...,0.11758,10.609865,0.0,0.000000,0.0,0.0,-0.500000,-0.650000,0.0,100.0
49996,1550149200,14.865600,-91.501999,2.0,10,mild temperate,2530.0,9.415068,11.627143,604.499035,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.672150,14.188867,0.0,0.0
49997,1553637300,37.688999,-121.442001,18.0,10,mild temperate,56.0,52.537946,12.823571,743.326800,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-2.497864,4.249994,-32.0,63.0
49998,1548885600,27.250999,2.512020,11.0,0,dry,269.0,-62.773181,17.074286,737.590379,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.235107,-5.885107,0.0,0.0


In [8]:
X_train = train.iloc[:,6:]
X_dev_in = dev_in.iloc[:,6:]
X_dev_out = dev_out.iloc[:,6:]
y_train = train['fact_temperature']
y_dev_in = dev_in['fact_temperature']
y_dev_out = dev_out['fact_temperature']

In [10]:
model = CatBoostRegressor(iterations=2000, learning_rate=0.05, depth=5)
# Fit model
model.fit(X_train,y_train)
# Get predictions
preds = model.predict(X_dev_in)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 9.8736442	total: 129ms	remaining: 4m 18s
1:	learn: 9.4482199	total: 255ms	remaining: 4m 14s
2:	learn: 9.0459713	total: 397ms	remaining: 4m 24s
3:	learn: 8.6645849	total: 682ms	remaining: 5m 40s
4:	learn: 8.3023627	total: 1.11s	remaining: 7m 22s
5:	learn: 7.9594050	total: 1.5s	remaining: 8m 18s
6:	learn: 7.6356939	total: 1.88s	remaining: 8m 54s
7:	learn: 7.3270001	total: 2.27s	remaining: 9m 25s
8:	learn: 7.0331777	total: 2.65s	remaining: 9m 45s
9:	learn: 6.7557230	total: 3.02s	remaining: 10m
10:	learn: 6.4929188	total: 3.39s	remaining: 10m 12s
11:	learn: 6.2425131	total: 3.74s	remaining: 10m 19s
12:	learn: 6.0058842	total: 4.1s	remaining: 10m 27s
13:	learn: 5.7843528	total: 4.51s	remaining: 10m 40s
14:	learn: 5.5713933	total: 4.89s	remaining: 10m 46s
15:	learn: 5.3705265	total: 5.27s	remaining: 10m 53s
16:	learn: 5.1815229	total: 5.56s	remaining: 10m 48s
17:	learn: 5.0043199	total: 5.68s	remaining: 10m 25s
18:	learn: 4.8353386	total: 5.84s	remaining: 10m 9s
19:	learn: 4.678699

158:	learn: 2.2072251	total: 36s	remaining: 6m 57s
159:	learn: 2.2061548	total: 36.4s	remaining: 6m 58s
160:	learn: 2.2054113	total: 36.7s	remaining: 6m 59s
161:	learn: 2.2043955	total: 37s	remaining: 6m 59s
162:	learn: 2.2036190	total: 37.4s	remaining: 7m 1s
163:	learn: 2.2029003	total: 37.8s	remaining: 7m 2s
164:	learn: 2.2023211	total: 38.2s	remaining: 7m 5s
165:	learn: 2.2014887	total: 38.6s	remaining: 7m 6s
166:	learn: 2.2006623	total: 39s	remaining: 7m 8s
167:	learn: 2.2001546	total: 39.2s	remaining: 7m 7s
168:	learn: 2.1993664	total: 39.6s	remaining: 7m 9s
169:	learn: 2.1988707	total: 40s	remaining: 7m 10s
170:	learn: 2.1980565	total: 40.3s	remaining: 7m 11s
171:	learn: 2.1971640	total: 40.6s	remaining: 7m 11s
172:	learn: 2.1966719	total: 41s	remaining: 7m 13s
173:	learn: 2.1959516	total: 41.3s	remaining: 7m 13s
174:	learn: 2.1954666	total: 41.6s	remaining: 7m 13s
175:	learn: 2.1946094	total: 41.7s	remaining: 7m 11s
176:	learn: 2.1938782	total: 41.8s	remaining: 7m 10s
177:	learn

315:	learn: 2.1316431	total: 1m 13s	remaining: 6m 30s
316:	learn: 2.1312805	total: 1m 13s	remaining: 6m 30s
317:	learn: 2.1308184	total: 1m 13s	remaining: 6m 30s
318:	learn: 2.1305983	total: 1m 14s	remaining: 6m 30s
319:	learn: 2.1303765	total: 1m 14s	remaining: 6m 30s
320:	learn: 2.1300346	total: 1m 14s	remaining: 6m 31s
321:	learn: 2.1295895	total: 1m 15s	remaining: 6m 32s
322:	learn: 2.1292397	total: 1m 15s	remaining: 6m 32s
323:	learn: 2.1288791	total: 1m 15s	remaining: 6m 32s
324:	learn: 2.1286579	total: 1m 16s	remaining: 6m 33s
325:	learn: 2.1283547	total: 1m 16s	remaining: 6m 34s
326:	learn: 2.1281099	total: 1m 16s	remaining: 6m 33s
327:	learn: 2.1277770	total: 1m 17s	remaining: 6m 34s
328:	learn: 2.1275497	total: 1m 17s	remaining: 6m 34s
329:	learn: 2.1272637	total: 1m 18s	remaining: 6m 35s
330:	learn: 2.1269510	total: 1m 18s	remaining: 6m 35s
331:	learn: 2.1266779	total: 1m 18s	remaining: 6m 35s
332:	learn: 2.1264069	total: 1m 19s	remaining: 6m 36s
333:	learn: 2.1260679	total:

469:	learn: 2.0931982	total: 1m 50s	remaining: 6m
470:	learn: 2.0930398	total: 1m 50s	remaining: 6m
471:	learn: 2.0928571	total: 1m 51s	remaining: 5m 59s
472:	learn: 2.0926426	total: 1m 51s	remaining: 5m 59s
473:	learn: 2.0924894	total: 1m 51s	remaining: 5m 58s
474:	learn: 2.0922870	total: 1m 51s	remaining: 5m 57s
475:	learn: 2.0921366	total: 1m 51s	remaining: 5m 57s
476:	learn: 2.0919709	total: 1m 51s	remaining: 5m 57s
477:	learn: 2.0917780	total: 1m 52s	remaining: 5m 57s
478:	learn: 2.0916785	total: 1m 52s	remaining: 5m 57s
479:	learn: 2.0914947	total: 1m 52s	remaining: 5m 57s
480:	learn: 2.0913594	total: 1m 53s	remaining: 5m 57s
481:	learn: 2.0911427	total: 1m 53s	remaining: 5m 58s
482:	learn: 2.0909914	total: 1m 54s	remaining: 5m 58s
483:	learn: 2.0908656	total: 1m 54s	remaining: 5m 58s
484:	learn: 2.0907299	total: 1m 54s	remaining: 5m 58s
485:	learn: 2.0905685	total: 1m 55s	remaining: 5m 58s
486:	learn: 2.0903542	total: 1m 55s	remaining: 5m 58s
487:	learn: 2.0901708	total: 1m 55s	

...

1385:	learn: 1.9957357	total: 5m 19s	remaining: 2m 21s
1386:	learn: 1.9956731	total: 5m 19s	remaining: 2m 21s
1387:	learn: 1.9956089	total: 5m 19s	remaining: 2m 20s
1388:	learn: 1.9955443	total: 5m 19s	remaining: 2m 20s
1389:	learn: 1.9954835	total: 5m 19s	remaining: 2m 20s
1390:	learn: 1.9954223	total: 5m 19s	remaining: 2m 20s
1391:	learn: 1.9953344	total: 5m 19s	remaining: 2m 19s
1392:	learn: 1.9952915	total: 5m 20s	remaining: 2m 19s
1393:	learn: 1.9952405	total: 5m 20s	remaining: 2m 19s
1394:	learn: 1.9951429	total: 5m 20s	remaining: 2m 18s
1395:	learn: 1.9950011	total: 5m 20s	remaining: 2m 18s
1396:	learn: 1.9949391	total: 5m 20s	remaining: 2m 18s
1397:	learn: 1.9948539	total: 5m 21s	remaining: 2m 18s
1398:	learn: 1.9947938	total: 5m 21s	remaining: 2m 18s
1399:	learn: 1.9947119	total: 5m 21s	remaining: 2m 17s
1400:	learn: 1.9946688	total: 5m 22s	remaining: 2m 17s
1401:	learn: 1.9945772	total: 5m 22s	remaining: 2m 17s
1402:	learn: 1.9945040	total: 5m 22s	remaining: 2m 17s
1403:	lear

1535:	learn: 1.9861091	total: 5m 54s	remaining: 1m 47s
1536:	learn: 1.9860527	total: 5m 55s	remaining: 1m 46s
1537:	learn: 1.9859837	total: 5m 55s	remaining: 1m 46s
1538:	learn: 1.9859160	total: 5m 55s	remaining: 1m 46s
1539:	learn: 1.9858498	total: 5m 55s	remaining: 1m 46s
1540:	learn: 1.9857591	total: 5m 55s	remaining: 1m 46s
1541:	learn: 1.9857054	total: 5m 56s	remaining: 1m 45s
1542:	learn: 1.9856370	total: 5m 56s	remaining: 1m 45s
1543:	learn: 1.9855605	total: 5m 56s	remaining: 1m 45s
1544:	learn: 1.9854838	total: 5m 56s	remaining: 1m 44s
1545:	learn: 1.9854318	total: 5m 56s	remaining: 1m 44s
1546:	learn: 1.9853730	total: 5m 56s	remaining: 1m 44s
1547:	learn: 1.9853257	total: 5m 56s	remaining: 1m 44s
1548:	learn: 1.9852586	total: 5m 56s	remaining: 1m 43s
1549:	learn: 1.9852009	total: 5m 56s	remaining: 1m 43s
1550:	learn: 1.9851536	total: 5m 57s	remaining: 1m 43s
1551:	learn: 1.9851138	total: 5m 57s	remaining: 1m 43s
1552:	learn: 1.9850280	total: 5m 57s	remaining: 1m 42s
1553:	lear

1685:	learn: 1.9770891	total: 6m 27s	remaining: 1m 12s
1686:	learn: 1.9770321	total: 6m 27s	remaining: 1m 11s
1687:	learn: 1.9769873	total: 6m 27s	remaining: 1m 11s
1688:	learn: 1.9769152	total: 6m 28s	remaining: 1m 11s
1689:	learn: 1.9768617	total: 6m 28s	remaining: 1m 11s
1690:	learn: 1.9768221	total: 6m 28s	remaining: 1m 11s
1691:	learn: 1.9767718	total: 6m 29s	remaining: 1m 10s
1692:	learn: 1.9767284	total: 6m 29s	remaining: 1m 10s
1693:	learn: 1.9766840	total: 6m 29s	remaining: 1m 10s
1694:	learn: 1.9766201	total: 6m 30s	remaining: 1m 10s
1695:	learn: 1.9765606	total: 6m 30s	remaining: 1m 9s
1696:	learn: 1.9765009	total: 6m 30s	remaining: 1m 9s
1697:	learn: 1.9764340	total: 6m 30s	remaining: 1m 9s
1698:	learn: 1.9763983	total: 6m 30s	remaining: 1m 9s
1699:	learn: 1.9763431	total: 6m 30s	remaining: 1m 8s
1700:	learn: 1.9763037	total: 6m 31s	remaining: 1m 8s
1701:	learn: 1.9762493	total: 6m 31s	remaining: 1m 8s
1702:	learn: 1.9761662	total: 6m 31s	remaining: 1m 8s
1703:	learn: 1.976

1838:	learn: 1.9687436	total: 7m 2s	remaining: 37s
1839:	learn: 1.9686911	total: 7m 3s	remaining: 36.8s
1840:	learn: 1.9686377	total: 7m 3s	remaining: 36.6s
1841:	learn: 1.9685882	total: 7m 3s	remaining: 36.4s
1842:	learn: 1.9685204	total: 7m 4s	remaining: 36.1s
1843:	learn: 1.9684667	total: 7m 4s	remaining: 35.9s
1844:	learn: 1.9684101	total: 7m 4s	remaining: 35.7s
1845:	learn: 1.9683519	total: 7m 5s	remaining: 35.5s
1846:	learn: 1.9682752	total: 7m 5s	remaining: 35.2s
1847:	learn: 1.9682259	total: 7m 5s	remaining: 35s
1848:	learn: 1.9681704	total: 7m 5s	remaining: 34.8s
1849:	learn: 1.9681425	total: 7m 5s	remaining: 34.5s
1850:	learn: 1.9680658	total: 7m 5s	remaining: 34.3s
1851:	learn: 1.9680189	total: 7m 5s	remaining: 34s
1852:	learn: 1.9679785	total: 7m 6s	remaining: 33.8s
1853:	learn: 1.9679403	total: 7m 6s	remaining: 33.6s
1854:	learn: 1.9679127	total: 7m 6s	remaining: 33.3s
1855:	learn: 1.9678691	total: 7m 6s	remaining: 33.1s
1856:	learn: 1.9678018	total: 7m 6s	remaining: 32.8s

1991:	learn: 1.9609792	total: 7m 36s	remaining: 1.83s
1992:	learn: 1.9609473	total: 7m 37s	remaining: 1.6s
1993:	learn: 1.9608816	total: 7m 37s	remaining: 1.38s
1994:	learn: 1.9608298	total: 7m 37s	remaining: 1.15s
1995:	learn: 1.9607763	total: 7m 38s	remaining: 918ms
1996:	learn: 1.9607283	total: 7m 38s	remaining: 689ms
1997:	learn: 1.9606660	total: 7m 38s	remaining: 459ms
1998:	learn: 1.9606178	total: 7m 39s	remaining: 230ms
1999:	learn: 1.9605695	total: 7m 39s	remaining: 0us


In [11]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X_train, y_train, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [12]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_dev_in, preds)

Test set evaluation:
_____________________________________
MAE: 1.4650720547008274
MSE: 3.898781078058175
RMSE: 1.9745331291366512
R2 Square 0.9633155532610546
__________________________________
